In [1]:
import typing as tp

In [2]:
class InfixToPostfixConverter:
    _OPERATORS_PRIORITY: dict[str, int] = {
        '+': 1, '-': 1,
        '*': 2, '/': 2, '%': 2,
        '<': 0, '>': 0, '<=': 0, '>=': 0, '==': 0, '!=': 0,
        '^': 3,
    }

    def infix_to_postfix(self, infix_expression: str) -> str:
        output: list[str] = []
        stack: list[str] = []
        tokens = infix_expression.split()

        for token in tokens:
            if self._is_number(token) or self._is_variable(token):
                output.append(token)
            elif token == '(':
                stack.append(token)
            elif token == ')':
                while stack and stack[-1] != '(':
                    output.append(stack.pop())
                if not stack:
                    raise ValueError("Ошибка: несогласованность скобок.")
                stack.pop()
            elif token in self._OPERATORS_PRIORITY:
                while stack and stack[-1] in self._OPERATORS_PRIORITY and \
                    self._OPERATORS_PRIORITY[stack[-1]] >= self._OPERATORS_PRIORITY[token]:
                    output.append(stack.pop())
                stack.append(token)
            else:
                raise ValueError(f"Неизвестный токен: {token}")

        while stack:
            top = stack.pop()
            if top in ('(', ')'):
                raise ValueError("Ошибка: несогласованность скобок.")
            output.append(top)

        return ' '.join(output)

    def evaluate_postfix(
        self,
        postfix_expression: str,
        variable_values: tp.Optional[tp.Mapping[str, int | float]] = None,
    ) -> int | float:
        stack: list[int | float] = []
        tokens = postfix_expression.split()
        variable_values = variable_values or {}

        for token in tokens:
            if self._is_number(token):
                stack.append(float(token))
            elif self._is_variable(token):
                if token not in variable_values:
                    raise ValueError(f"Переменная '{token}' не определена.")
                stack.append(float(variable_values[token]))
            elif token in self._OPERATORS_PRIORITY:
                if len(stack) < 2:
                    raise ValueError("Ошибка вычисления: недостаточно операндов.")
                b = stack.pop()
                a = stack.pop()
                result = self._apply_operator(a, b, token)
                stack.append(result)
            else:
                raise ValueError(f"Неизвестный токен в постфиксной записи: {token}")

        if len(stack) != 1:
            raise ValueError("Ошибка вычисления: итоговое значение не единственное.")

        return stack[0]

    @classmethod
    def _apply_operator(
        cls,
        a: float,
        b: float,
        operator: str
    ) -> float:
        match operator:
            case '+':
                return a + b
            case '-':
                return a - b
            case '*':
                return a * b
            case '/':
                if b == 0:
                    raise ZeroDivisionError("Деление на ноль.")
                return a / b
            case '%':
                return a % b
            case '^':
                return a ** b
            case '<':
                return float(a < b)
            case '>':
                return float(a > b)
            case '<=':
                return float(a <= b)
            case '>=':
                return float(a >= b)
            case '==':
                return float(a == b)
            case '!=':
                return float(a != b)
            case _:
                raise ValueError(f"Не поддерживаемый оператор: {operator}")

    @classmethod
    def _is_number(cls, token: str) -> bool:
        try:
            float(token)
            return True
        except ValueError:
            return False

    @classmethod
    def _is_variable(cls, token: str) -> bool:
        return token[0].isalpha() or token[0] == '_'

In [3]:
converter = InfixToPostfixConverter()

In [4]:
test_expressions: list[str] = [
    "3 + 4 * ( 2 - ( 7 + 1 ) ) ^ 2",
    "a + b * ( c - d ) / e - f ^ g",
    "( 3 + 4 ) * ( ( 2 - 7 ) / ( 5 - ( 6 + 1 ) ) ) ^ ( 2 + 1 )",
    "-3.5 + 2.5 * ( 4 - 6.0 ) / 2 ^ 2",
    "a % 3 + ( b - 4 ) * ( c + 2 ) / ( d - 1 ) ^ 2",
]

In [5]:
variable_mappings: dict[int, dict[str, float]] = {
    2: {
        'a': 2,
        'b': 3,
        'c': 10,
        'd': 4,
        'e': 2,
        'f': 2,
        'g': 3
    },
    5: {
        'a': 10,
        'b': 7,
        'c': 3,
        'd': 5
    },
}

In [6]:
for i, expression in enumerate(test_expressions, start=1):
    print(f"--- Тестовое выражение {i} ---")
    print("Инфиксная запись:")
    print(expression)

    try:
        postfix = converter.infix_to_postfix(expression)
        print("Постфиксная запись:")
        print(postfix)
    except Exception as e:
        print("Ошибка при преобразовании в постфиксную запись:", e)
        continue

    if any(ch.isalpha() for ch in expression):
        mapping = variable_mappings.get(i, {})
        print("Используемые переменные:", mapping)
        try:
            result = converter.evaluate_postfix(postfix, mapping)
            print("Вычисленный результат:")
            print(result)
        except Exception as e:
            print("Ошибка при вычислении постфиксного выражения:", e)
    else:
        try:
            result = converter.evaluate_postfix(postfix)
            print("Вычисленный результат:")
            print(result)
        except Exception as e:
            print("Ошибка при вычислении постфиксного выражения:", e)
    
    print()

--- Тестовое выражение 1 ---
Инфиксная запись:
3 + 4 * ( 2 - ( 7 + 1 ) ) ^ 2
Постфиксная запись:
3 4 2 7 1 + - 2 ^ * +
Вычисленный результат:
147.0

--- Тестовое выражение 2 ---
Инфиксная запись:
a + b * ( c - d ) / e - f ^ g
Постфиксная запись:
a b c d - * e / + f g ^ -
Используемые переменные: {'a': 2, 'b': 3, 'c': 10, 'd': 4, 'e': 2, 'f': 2, 'g': 3}
Вычисленный результат:
3.0

--- Тестовое выражение 3 ---
Инфиксная запись:
( 3 + 4 ) * ( ( 2 - 7 ) / ( 5 - ( 6 + 1 ) ) ) ^ ( 2 + 1 )
Постфиксная запись:
3 4 + 2 7 - 5 6 1 + - / 2 1 + ^ *
Вычисленный результат:
109.375

--- Тестовое выражение 4 ---
Инфиксная запись:
-3.5 + 2.5 * ( 4 - 6.0 ) / 2 ^ 2
Постфиксная запись:
-3.5 2.5 4 6.0 - * 2 2 ^ / +
Вычисленный результат:
-4.75

--- Тестовое выражение 5 ---
Инфиксная запись:
a % 3 + ( b - 4 ) * ( c + 2 ) / ( d - 1 ) ^ 2
Постфиксная запись:
a 3 % b 4 - c 2 + * d 1 - 2 ^ / +
Используемые переменные: {'a': 10, 'b': 7, 'c': 3, 'd': 5}
Вычисленный результат:
1.9375

